In [1]:
import numpy as np
import pandas as pd 
import math as m

#parametri GRS80 elipsoida
a = 6378137
b = 6356752.3141
e2 = 0.00669438002290
eprim2 = 0.00673949677548

In [2]:
def dms2degree(deg):
    d = deg[0]+deg[1]/60+deg[2]/3600
    if d < 0:
        d +=360
    return d

In [3]:
def degree2dms(a):
    minuta = (a%1)*60
    sekunda =(minuta%1)*60
    
    return[int(a),int(minuta),round(sekunda,5)]

In [4]:
def dms2rad(deg):
    return np.radians(dms2degree(deg))

def deg2rad(deg):
    return np.radians(deg)

In [5]:
def redukcijaPravaca(koor1,koor2,duzina,pravac,azimut,ksi,eta):
   
    
    delta_h = koor2[2]-koor1[2]
    Bm = (koor1[0]+koor2[0])/2
    
    c1 = -1*((duzina**2*e2)/(12*a**2))*m.cos(Bm)**2 * m.sin(2*azimut)
    c2 = (koor1[2]*e2)/(2*a)*m.cos(Bm)**2*m.sin(2*azimut)
    c3 = (eta*m.cos(azimut) - ksi*m.sin(azimut))*delta_h/duzina
    delta = c1+c2+c3
    p = pravac+delta
    
    
    return p


In [6]:
def redukcijaHorizont(koor1,koor2,koor3,duzina1,pravac1,azimut1,eta1,ksi1,duzina2,pravac2,azimut2,eta2,ksi2):

    pravac_1 = redukcijaPravaca(koor1,koor2,duzina1,pravac1,azimut1,eta1,ksi1)
    pravac_2 = redukcijaPravaca(koor1,koor3,duzina2,pravac2,azimut2,eta2,ksi2)

    return pravac_2 -pravac_1;



In [7]:
def redukcijaDuzina(koor1,koor2, Do, azimut):
    
    deltah= koor2[2] - koor1[2]
    
    phi1 = koor1[0]
    phi2 = koor2[0]
    Bm= (phi1 + phi2) / 2
    M = (a * (1 - e2)) / (m.sqrt(m.pow((1 - e2 * m.pow(m.sin(Bm) ,2)), 3)))
    N = a / (m.sqrt(1-e2*m.pow(m.sin(Bm),2)))
    RA = (M * N) / (N * m.pow(m.cos(azimut), 2) + M * m.pow(m.sin(azimut), 2))
    Dprim = m.sqrt((m.pow(Do, 2) - m.pow(deltah, 2)) / ((1 + koor1[2] / RA) * (1 + koor2[2] / RA)))
    s = 2 * RA * m.asin(Dprim / (2 * RA))
    return s - (m.pow(e2, 2) * m.pow(s, 5) * m.pow((m.cos(Bm)), 4) * m.pow((m.sin(2 * azimut)), 2)) / (360 * m.pow(a,4))

In [8]:
def preprocessData(data):
    data = data.drop(0,axis=0)
    data = data.rename(columns ={data.columns[3]:'B_min','Unnamed: 4':'B_sec','Unnamed: 6':'L_min','Unnamed: 7':'L_sec',data.columns[9]:'kosa_duz'})
    data = data.astype(float)
    data[['Stanica','Vizura','B','B_min','L_min','L']] = data[['Stanica','Vizura','B','B_min','L_min','L']].astype(int)
    koordinate = data.groupby('Stanica').first().loc[:,['B','B_min','B_sec','L','L_min','L_sec','H']]
    koordinate['phi_dms'] =koordinate[['B','B_min','B_sec']].values.tolist()
    koordinate['lambda_dms'] =koordinate[['L','L_min','L_sec']].values.tolist()
    koordinate['phi']=koordinate['phi_dms'].apply(dms2degree).apply(deg2rad)
    koordinate['lambda']=koordinate['lambda_dms'].apply(dms2degree).apply(deg2rad)
    koor = koordinate[['phi','lambda','H']].copy()
    
    podaci_1 = data.loc[:,['Stanica','Vizura','Pravac','Azimut','Ksi','Eta','kosa_duz']].copy()
    podaci_1['azimut'] = data['Azimut'].apply(deg2rad)
    podaci_1['pravac'] = data['Pravac'].apply(deg2rad)
    podaci_1['ksi'] = (data['Ksi']/3600).apply(deg2rad)
    podaci_1['eta'] = (data['Eta']/3600).apply(deg2rad)
    merenja = podaci_1.drop(['Pravac','Azimut','Ksi','Eta',],axis=1).copy()
    merenja =merenja.set_index(['Stanica','Vizura'])
    merenja = merenja[['kosa_duz', 'pravac','azimut', 'ksi', 'eta']]
    
    
    
    return koor,merenja

In [9]:
def redukcijaMerenja(merenja,koor):
    duzine = []
    pravci = []
    for a,b in merenja.index:
        duzina = redukcijaDuzina(koor.loc[a],koor.loc[b],merenja.loc[(a,b),'kosa_duz'],merenja.loc[(a,b),'azimut'])
        pravac = redukcijaPravaca(koor.loc[a],koor.loc[b],merenja.loc[(a,b),'kosa_duz']
                                  ,merenja.loc[(a,b),'pravac'],merenja.loc[(a,b),'azimut'],
                                  merenja.loc[(a,b),'ksi'],merenja.loc[(a,b),'eta'])
        duzine.append(duzina)
        pravci.append(pravac)


    idx_resenja = merenja.index
    resenja = pd.DataFrame({'Redukovane duzine':duzine,'Redukovani pravci':pravci},index =idx_resenja )
    resenja[['Stepen','Minut','Sekund']] = np.degrees(resenja['Redukovani pravci']).apply(degree2dms).values.tolist()
    index = pd.MultiIndex.from_product([['Redukovani pravac'],['Stepen','Minut','Sekund']])
    izlaz = pd.DataFrame(resenja,columns = index)
    izlaz['Redukovane duzine'] = resenja['Redukovane duzine']
    izlaz['Redukovani pravac'] = resenja[['Stepen','Minut','Sekund']]
    izlaz[('Redukovani pravac','Stepen')] = izlaz[('Redukovani pravac','Stepen')].astype(int)
    izlaz[('Redukovani pravac','Minut')] = izlaz[('Redukovani pravac','Minut')].astype(int)
    return resenja.drop(['Stepen','Minut','Sekund'],axis=1),izlaz

In [10]:
def horizontalniUglovi(res):
    rez= []
    for stanica, df in res.groupby(level=0):
        df = res.loc[stanica]
        for vizura in range(len(df.index)):
            for vizura_2 in range(vizura+1,len(df.index)):
                ugao= res.loc[(stanica)].iloc[vizura_2,1]-res.loc[(stanica)].iloc[vizura,1]
                rez.append([stanica,res.loc[(stanica)].iloc[vizura].name,res.loc[(stanica)].iloc[vizura_2].name,ugao])

    horizontalni_uglovi = pd.DataFrame(rez)
    horizontalni_uglovi.columns = ['Stanica','Levi pravac','Desni pravac','Ugao']
    horizontalni_uglovi.set_index(['Stanica','Levi pravac','Desni pravac'],inplace= True)
    horizontalni_uglovi[['Stepen','Minut','Sekund']] = np.degrees(horizontalni_uglovi['Ugao']).apply(degree2dms).values.tolist()
    horizontalni_uglovi[['Stepen','Minut']]=horizontalni_uglovi[['Stepen','Minut']].astype(np.int32).copy()
    return horizontalni_uglovi


In [11]:
def poredjenjeHorizonta(uglovi,redukovani):
    index = pd.MultiIndex.from_product([['Redukovani_Ugao','Ugao'],['Stepen','Minut','Sekund']])
    horizon = pd.DataFrame(redukovani,columns = index)
    horizon['Redukovani_Ugao'] = redukovani[['Stepen','Minut','Sekund']]
    horizon['Ugao'] = uglovi[['Stepen','Minut','Sekund']]
    horizon[('Redukovani_Ugao','Stepen')] = horizon[('Redukovani_Ugao','Stepen')].astype(int)
    horizon[('Redukovani_Ugao','Minut')] = horizon[('Redukovani_Ugao','Minut')].astype(int)
    horizon[('Ugao','Stepen')] = horizon[('Ugao','Stepen')].astype(int)
    horizon[('Ugao','Minut')] = horizon[('Ugao','Minut')].astype(int)
    return horizon

In [12]:
def resenjeZadatka(data):
    koor,merenja = preprocessData(data)
    res,izlaz = redukcijaMerenja(merenja,koor)
    hori = horizontalniUglovi(merenja)
    hori_redu = horizontalniUglovi(res)
    predj = poredjenjeHorizonta(hori,hori_redu)
    dfs = {'Koordinate tacaka':koor, 'Izvrsena merenja':merenja,'Redukovana merenje':res,'Horizontalni uglovi':predj}
    writer = pd.ExcelWriter('Izvrsene redukcije.xlsx', engine='xlsxwriter')
    for sheet_name in dfs.keys():
        dfs[sheet_name].to_excel(writer, sheet_name=sheet_name)

    writer.save()
    
    return koor,merenja,izlaz,predj

In [15]:
data = pd.read_excel(r"TestPodaci_dodatniZadatak.xlsx")

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Stanica       10 non-null     float64
 1   Vizura        10 non-null     float64
 2   B             11 non-null     object 
 3   Unnamed: 3    11 non-null     object 
 4   Unnamed: 4    11 non-null     object 
 5   L             11 non-null     object 
 6   Unnamed: 6    11 non-null     object 
 7   Unnamed: 7    11 non-null     object 
 8   H             11 non-null     object 
 9   Kosa duzina   11 non-null     object 
 10  Pravac        11 non-null     object 
 11  Azimut        11 non-null     object 
 12  Ksi           11 non-null     object 
 13  Eta           11 non-null     object 
dtypes: float64(2), object(12)
memory usage: 1.3+ KB


In [17]:
data

,Stanica,Vizura,B,Unnamed: 3,Unnamed: 4,L,Unnamed: 6,Unnamed: 7,H,Kosa duzina,Pravac,Azimut,Ksi,Eta
0,NaN,NaN,°,¢,²,°,¢,²,[m],[m],[dec],[dec],²,²
1,120.0,50.0,44,39,42.58909,20,30,29.64214,191.53,5920.965,0,268.276733,1.05,3.15
2,120.0,25.0,44,39,42.58909,20,30,29.64214,191.53,7972.146,87.534359,355.811092,1.12,3.08
3,120.0,15.0,44,39,42.58909,20,30,29.64214,191.53,4977.861,151.462225,59.73896,1.08,3.09
4,15.0,120.0,44,41,4.65815,20,33,44.21837,253.37,4977.864,0,239.73896,1.09,3.14
5,15.0,50.0,44,41,4.65815,20,33,44.21837,253.37,10562.878,15.529695,255.265955,1.07,3.1
6,15.0,25.0,44,41,4.65815,20,33,44.21837,253.37,7310.424,78.365885,318.102145,1.1,3.13
7,50.0,25.0,44,39,35.5782,20,26,1.1255,366.06,9788.726,0,33.076942,1.09,3.1
8,50.0,120.0,44,39,35.5782,20,26,1.1255,366.06,5920.969,55.001697,88.276733,1.06,3.09
9,25.0,50.0,44,44,0.0396,20,30,0.9958,352.66,9788.693,0,213.076942,0.9,3.1


In [18]:
koordinate, merenja, redukovana_merenja, horizontalni = resenjeZadatka(data)

C:\Users\ilija\AppData\Local\Temp\ipykernel_15468\3975370455.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  deltah= koor2[2] - koor1[2]
C:\Users\ilija\AppData\Local\Temp\ipykernel_15468\3975370455.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  phi1 = koor1[0]
C:\Users\ilija\AppData\Local\Temp\ipykernel_15468\3975370455.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  phi2 = koor2[0]
C:\Users\ilija\AppData\Local\Temp\ipy

ModuleNotFoundError: No module named 'xlsxwriter'

In [ ]:
koordinate

,phi,lambda,H
Stanica,,,
15,0.779894,0.358880,253.37
25,0.780744,0.357797,352.66
50,0.779462,0.356634,366.06
120,0.779496,0.357936,191.53


In [ ]:
merenja

kosa_duz    pravac    azimut       ksi       eta
Stanica Vizura                                                   
120     50       5920.965  0.000000  4.682312  0.000005  0.000015
        25       7972.146  1.527763  6.210075  0.000005  0.000015
        15       4977.861  2.643515  1.042642  0.000005  0.000015
15      120      4977.864  0.000000  4.184234  0.000005  0.000015
        50      10562.878  0.271044  4.455231  0.000005  0.000015
        25       7310.424  1.367743  5.551930  0.000005  0.000015
50      25       9788.726  0.000000  0.577302  0.000005  0.000015
        120      5920.969  0.959961  1.540720  0.000005  0.000015
25      50       9788.693  0.000000  3.718894  0.000004  0.000015
        120      9788.693  0.653869  3.065025  0.000004  0.000015

In [ ]:
horizontalni

Redukovani_Ugao                   Ugao        \
                                          Stepen Minut    Sekund Stepen Minut   
Stanica Levi pravac Desni pravac                                                
15      120         50                        15    31  46.89137     15    31   
                    25                        78    21  57.19392     78    21   
        50          25                        62    50  10.30255     62    50   
25      50          120                       37    27  50.23680     37    27   
50      25          120                       55     0   6.12345     55     0   
120     50          25                        87    32   3.72585     87    32   
                    15                       151    27  43.99808    151    27   
        25          15                        63    55  40.27223     63    55   

                                           
                                   Sekund  
Stanica Levi pravac Desni pravac           
15      120         50            46.9020  
                    25            57.1860  
        50          25            10.2840  
25      50          120           50.2020  
50      25          120            6.1092  
120     50          25             3.6924  
                    15            44.0100  
        25          15            40.3176

In [ ]:
redukovana_merenja

Redukovani pravac                 Redukovane duzine
                          Stepen Minut    Sekund                  
Stanica Vizura                                                    
120     50                     0     0   0.02877       5918.134116
        25                    87    32   3.75463       7970.177407
        15                   151    27  44.02685       4977.303536
15      120                    0     0   0.02001       4977.306536
        50                    15    31  46.91137      10561.765939
        25                    78    21  57.21392       7309.402756
50      25                     0     0   0.01545       9788.165898
        120                   55     0   6.13889       5918.138118
25      50                     0     0   0.01463       9788.132900
        120                   37    27  50.25143       9786.949472